# Modeling

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Introduction" data-toc-modified-id="Introduction-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Introduction</a></span></li><li><span><a href="#Preprocess-Data-for-Modeling" data-toc-modified-id="Preprocess-Data-for-Modeling-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Preprocess Data for Modeling</a></span></li><li><span><a href="#Define-Functions-for-Model-Selection" data-toc-modified-id="Define-Functions-for-Model-Selection-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Define Functions for Model Selection</a></span></li><li><span><a href="#Model-Selection" data-toc-modified-id="Model-Selection-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Model Selection</a></span><ul class="toc-item"><li><span><a href="#Linear-Regression" data-toc-modified-id="Linear-Regression-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Linear Regression</a></span></li><li><span><a href="#Decision-Tree-Regressor" data-toc-modified-id="Decision-Tree-Regressor-4.2"><span class="toc-item-num">4.2&nbsp;&nbsp;</span>Decision Tree Regressor</a></span></li><li><span><a href="#Random-Forest-Regressor" data-toc-modified-id="Random-Forest-Regressor-4.3"><span class="toc-item-num">4.3&nbsp;&nbsp;</span>Random Forest Regressor</a></span></li><li><span><a href="#Random-Forest-Regressor-–-Grid-Search" data-toc-modified-id="Random-Forest-Regressor-–-Grid-Search-4.4"><span class="toc-item-num">4.4&nbsp;&nbsp;</span>Random Forest Regressor – Grid Search</a></span></li></ul></li><li><span><a href="#Feature-Importance" data-toc-modified-id="Feature-Importance-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Feature Importance</a></span></li><li><span><a href="#Final-Model" data-toc-modified-id="Final-Model-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Final Model</a></span><ul class="toc-item"><li><span><a href="#Findings" data-toc-modified-id="Findings-6.1"><span class="toc-item-num">6.1&nbsp;&nbsp;</span>Findings</a></span></li></ul></li></ul></div>

## Introduction

In this notebook, we'll be prepping the data for modeling and creating the following:
* Pipeline for numerical features
    * Scale data
    * Impute missing values
* Pipeline for categorical features
   * One-hot encode some features
   * Use a count vectorizer for 'AdminUnit'
* Column transformer to transform our training set
* Two functions for model selection

We'll also be taking a look a **feature importance** and fine-tuning our best model.

In [53]:
from pprint import pprint

import joblib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pickle

from scipy import stats
from sklearn.base import BaseEstimator
from sklearn.base import TransformerMixin
from sklearn.compose import make_column_transformer
from sklearn.ensemble import RandomForestRegressor
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeRegressor

# display estimators as a diagram
from sklearn import set_config
set_config(display='diagram')

# display plots inline
%matplotlib inline

In [2]:
# load pickle
df = pd.read_pickle('../data/01_fire_weather.pkl')
df.head()

,Name,Latitude,Longitude,AcresBurned,Counties,AdminUnit,Updated,Started,Extinguished,CalFireIncident,...,geometry,RequestURL,sunsetTime,moonPhase,precipIntensity,temperatureHigh,temperatureLow,dewPoint,humidity,windSpeed
52378c,Pilot Fire,40.618056,-123.675556,30,[Humboldt],CAL FIRE Humbolt-Del Norte Unit,2019-01-15 10:38:00+00:00,2019-01-01 14:14:00+00:00,2019-01-15 10:38:00+00:00,True,...,POINT (-123.67556 40.61806),https://api.darksky.net/forecast/4f4ef1267ba70...,1546390740,0.88,0.0006,52.46,32.19,13.31,0.37,5.86
398fe2,Scott Fire,39.246780,-121.323990,80,[Yuba],CAL FIRE Nevada-Yuba-Placer Unit,2019-06-27 08:24:51.860000+00:00,2019-06-25 13:51:09+00:00,2019-06-25 13:50:00+00:00,True,...,POINT (-121.32399 39.24678),https://api.darksky.net/forecast/4f4ef1267ba70...,1561520220,0.77,0.0004,91.96,60.33,50.88,0.42,4.96
030e7d,York Fire,35.817780,-120.097150,161,[Kings],Fresno-Kings County,2019-06-24 11:45:00+00:00,2019-04-28 22:20:00+00:00,2019-06-24 11:45:00+00:00,True,...,POINT (-120.09715 35.81778),https://api.darksky.net/forecast/4f4ef1267ba70...,1556505960,0.82,0.0009,83.37,56.30,48.51,0.55,7.75
b5b503,Cana Fire,39.839580,-121.957000,10,[Butte],CAL FIRE / Butte County Fire,2019-05-08 08:47:00+00:00,2019-04-30 12:20:00+00:00,2019-05-08 08:47:00+00:00,True,...,POINT (-121.95700 39.83958),https://api.darksky.net/forecast/4f4ef1267ba70...,1556679720,0.88,0.0016,72.21,47.45,45.50,0.60,6.20
abd7ac,Woody Fire,35.651890,-118.925450,115,[Kern],Kern County Fire Department,2019-05-08 08:47:00+00:00,2019-04-30 18:37:00+00:00,2019-05-08 08:47:00+00:00,False,...,POINT (-118.92545 35.65189),https://api.darksky.net/forecast/4f4ef1267ba70...,1556678520,0.88,0.0007,70.64,51.38,46.74,0.63,6.10


## Preprocess Data for Modeling

In [3]:
def categorize_dates():
    """Cast dates features as 'category' type."""
    df['StartedYear'] = df['Started'].dt.year.astype('category')
    df['StartedMonth'] = df['Started'].dt.month_name().astype('category')

categorize_dates()

In [4]:
# define DataFrame with only numerical features
num_df = df.select_dtypes(include='number').drop(columns='AcresBurned')

# define numerical columns
num_cols = num_df.columns

# define non-numerical columns
cols = ['AdminUnit', 'CalFireIncident',
        'StartedYear', 'StartedMonth']

# extend 'cols' to include numerical
cols.extend(num_df.columns)
cols

['AdminUnit',
 'CalFireIncident',
 'StartedYear',
 'StartedMonth',
 'Latitude',
 'Longitude',
 'sunsetTime',
 'moonPhase',
 'precipIntensity',
 'temperatureHigh',
 'temperatureLow',
 'dewPoint',
 'humidity',
 'windSpeed']

In [5]:
# define X and y
X = df.loc[:, cols]
y = df['AcresBurned'].copy()

# train test split
X_tr, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [6]:
# construct pipeline for numerical attributes
num_pipeline = make_pipeline(SimpleImputer(), StandardScaler())

# construct column transformer
col_tr = make_column_transformer(
    (CountVectorizer(), 'AdminUnit'),
    (OneHotEncoder(handle_unknown='ignore'),
     ['CalFireIncident', 'StartedYear', 'StartedMonth']),
    (num_pipeline, num_cols),
)

# display pipeline diagram
col_tr

ColumnTransformer(transformers=[('countvectorizer', CountVectorizer(),
                                 'AdminUnit'),
                                ('onehotencoder',
                                 OneHotEncoder(handle_unknown='ignore'),
                                 ['CalFireIncident', 'StartedYear',
                                  'StartedMonth']),
                                ('pipeline',
                                 Pipeline(steps=[('simpleimputer',
                                                  SimpleImputer()),
                                                 ('standardscaler',
                                                  StandardScaler())]),
                                 Index(['Latitude', 'Longitude', 'sunsetTime', 'moonPhase', 'precipIntensity',
       'temperatureHigh', 'temperatureLow', 'dewPoint', 'humidity',
       'windSpeed'],
      dtype='object'))])

In [7]:
# transform X training set
X_train = col_tr.fit_transform(X_tr)

Since we'll be testing several models, let's define a function `train_model` that makes a pipeline, trains the model, and prints regression metrics.

## Define Functions for Model Selection

Since we'll be testing several models, let's define a function `train_model` that makes a pipeline, trains the model, and prints regression metrics.

We'll also define the function `cv_rmse_scores` to cross-validate a model and print RMSE scores.

In [8]:
def train_model(model, X=X_train):
    """Input model, print scores, return trained model."""
    # train model
    model.fit(X, y_train)

    # make predictions
    y_pred = model.predict(X)

    # Mean Absolute Error
    mae = mean_absolute_error(y_train, y_pred)
    print(f'MAE = {int(mae)}')

    # Mean Squared Error
    mse = mean_squared_error(y_train, y_pred)
    # print(f'MSE = {int(mse)}')

    # Root Mean Squared Error
    rmse = np.sqrt(mse)
    print(f'RMSE = {int(rmse)}')

    # R^2 Metric
    r2 = r2_score(y_train, y_pred)
    print(f'R^2 = {round(r2, 3)}')

    return model

In [9]:
# performance dictionary for model comparison
training_model_performance = {}

def cv_rmse_scores(model_name, model, X=X_train):
    """Cross-validates model and prints RMSE scores.
    
    :param model_name: Name of the model to be used for comparison.
    :param model: Variable of the model that will be cross-validated.
    :param X: Defaults to X_train.
    """
    # get cross-validation MSE score
    scores = cross_val_score(model, X, y_train,
                             scoring="neg_mean_squared_error", cv=10,
                             n_jobs=-1)
    
    # get RMSE by taking the square root of MSE
    rmse_scores = np.sqrt(-scores)
    mean = int(rmse_scores.mean())
    std = int(rmse_scores.std())
    
    
    # print score
    print('RMSE Scores - 10 fold')
    print('* mean:', mean)
    print('* std:', std)

    # add score to performance dict
    training_model_performance[model_name] = {'rmse': mean, 'std': std}

## Model Selection

We'll be training and comparing three models, including:
* Linear Regression
* Decision Tree Regressor
* Random Forest Regressor

After comparison, we'll use grid search to tune the hyperparameters.

### Linear Regression

In [10]:
# instantiate linear regression base model 
lin_reg = LinearRegression()

# train model and print scores 
train_model(lin_reg);

MAE = 5472
RMSE = 15214
R^2 = 0.396


In [11]:
# cross-validation
cv_rmse_scores('Linear Regression', lin_reg)

RMSE Scores - 10 fold
* mean: 19726
* std: 8832


### Decision Tree Regressor

Let’s train a **Decision Tree Regressor**. This is a powerful model, capable of finding complex nonlinear relationships in the data

In [12]:
# instantiate decision tree base model
tree_reg = DecisionTreeRegressor(random_state=1)

# train model and print scores
train_model(tree_reg);

MAE = 0
RMSE = 0
R^2 = 1.0


Wait, what!? No error at all? Could this model really be absolutely perfect? Of course, it is much more likely that the model has badly overfit the data.

Let's use Scikit-Learn's `cross-validation` to evaluate.

In [13]:
# cross-validation
cv_rmse_scores('Decison Tree', tree_reg)

RMSE Scores - 10 fold
* mean: 24758
* std: 14149


Now the Decision Tree doesn’t look as good as it did earlier. In fact, it seems to perform worse than the Linear Regression model!

### Random Forest Regressor

In [14]:
# instantiate random forest base model
forest_reg = RandomForestRegressor(random_state=1)

# train model and print scores
train_model(forest_reg);

MAE = 1850
RMSE = 8350
R^2 = 0.818


In [15]:
# cross-validation
cv_rmse_scores('Random Forest', forest_reg)

RMSE Scores - 10 fold
* mean: 18061
* std: 9984


The Random Forest Regressor performed the best so we'll be tuning the hyperparameters to improve its performance.

### Random Forest Regressor – Grid Search 

In [18]:
# define grid
param_grid = {
    'n_estimators': [200, 300, 400, 500],
    'max_features': ['sqrt'],
    'max_depth': [90, 100, 110, 120],
    'min_samples_split': [1, 2, 3],
    'min_samples_leaf': [1, 2, 3],
    'bootstrap': [True],
}

# find best hyperparameters
rf_grid = GridSearchCV(
    estimator=forest_reg,
    param_grid=param_grid,
    cv=5,
    scoring='neg_mean_squared_error',
    n_jobs=-1
)

# fit the grid model
rf_grid.fit(X_train, y_train)

# view best parameters
rf_grid.best_params_

{'bootstrap': True,
 'max_depth': 90,
 'max_features': 'sqrt',
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'n_estimators': 200}

In [19]:
cv_rmse_scores('Random Forest - Grid', rf_grid.best_estimator_)

RMSE Scores - 10 fold
* mean: 16029
* std: 10302


In [24]:
# compare model performance
pd.DataFrame.from_dict(training_model_performance, orient='index').sort_values('rmse')

,rmse,std
Random Forest - Grid,16029,10302
Random Forest,18061,9984
Linear Regression,19726,8832
Decison Tree,24758,14149


## Feature Importance

In [25]:
feature_importances = rf_grid.best_estimator_.feature_importances_

In [26]:
attributes = []

# get count vectorizer attributes
ctv = col_tr.named_transformers_['countvectorizer']
ctv_attr = ctv.get_feature_names()
attributes.extend(ctv_attr)

# get one hot encoder attributes
ohe = col_tr.named_transformers_['onehotencoder']
for lst in ohe.categories_:
    attributes.extend(lst)

# add nummerical attributes
attributes.extend(num_cols)

In [27]:
# show top 20 features
sorted(zip(feature_importances, attributes), reverse=True)[:20]

[(0.06756729310263114, 'temperatureHigh'),
 (0.054863517114637855, 'national'),
 (0.049454233058986816, 'Longitude'),
 (0.04928099867022682, 'windSpeed'),
 (0.04558114408915441, 'dewPoint'),
 (0.04550493772876473, 'mendocino'),
 (0.0429214779803063, 'sunsetTime'),
 (0.042853617898527326, 'Latitude'),
 (0.04009326059906519, 'moonPhase'),
 (0.03967647746319461, 'temperatureLow'),
 (0.034937593344242346, 'humidity'),
 (0.03310273407143319, 'precipIntensity'),
 (0.03284155611952913, 'area'),
 (0.024317457245231853, 'yosemite'),
 (0.021647703540426937, 'sierra'),
 (0.02137865137869709, False),
 (0.02035731699595511, 'forest'),
 (0.020171420263523733, 'usfs'),
 (0.016964888992012574, 'July'),
 (0.016805207805763837, 'park')]

In [37]:
def indices_of_top_k(arr, k):
    return np.sort(np.argpartition(np.array(arr), -k)[-k:])

class TopFeatureSelector(BaseEstimator, TransformerMixin):
    """Feature selector class to select top features."""

    def __init__(self, feature_importances, k):
        self.feature_importances = feature_importances
        self.k = k

    def fit(self, X, y=None):
        self.feature_indices_ = indices_of_top_k(self.feature_importances, self.k)
        return self

    def transform(self, X):
        return X[:, self.feature_indices_]

Let's first try including the top 20 features and compare.

In [38]:
# define k
k = 20

# sort feature importances
sorted(zip(feature_importances, attributes), reverse=True)[:k];

Let's also define our full pipeline which includes a column transformer, feature selection, and estimator.

In [45]:
# define pipeline
full_pipeline = Pipeline([
    ('col_transformer', col_tr),
    ('feature_selection', TopFeatureSelector(feature_importances, k)),
    ('rf_reg', RandomForestRegressor(**rf_grid.best_params_)),
])

# display pipeline diagram
full_pipeline

Pipeline(steps=[('col_transformer',
                 ColumnTransformer(transformers=[('countvectorizer',
                                                  CountVectorizer(),
                                                  'AdminUnit'),
                                                 ('onehotencoder',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['CalFireIncident',
                                                   'StartedYear',
                                                   'StartedMonth']),
                                                 ('pipeline',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer()),
                                                                  ('standardscaler',
                                                                   StandardScaler())]),
                                                  Index(['Latitude', 'Longitud...
       7.43068918e-03, 5.21670928e-04, 5.61845250e-07, 5.92402461e-06,
       1.69648890e-02, 4.34380728e-03, 3.82210621e-06, 7.62437083e-04,
       1.77831343e-03, 1.20546879e-03, 3.05456670e-03, 4.28536179e-02,
       4.94542331e-02, 4.29214780e-02, 4.00932606e-02, 3.31027341e-02,
       6.75672931e-02, 3.96764775e-02, 4.55811441e-02, 3.49375933e-02,
       4.92809987e-02]),
                                    k=20)),
                ('rf_reg',
                 RandomForestRegressor(max_depth=90, max_features='sqrt',
                                       n_estimators=200))])

In [40]:
# train model and print scores
train_model(full_pipeline, X_tr);

MAE = 1781
RMSE = 7441
R^2 = 0.855


In [41]:
# cross-validation
cv_rmse_scores('Random Forest - K20', full_pipeline.named_steps.rf_reg, X_train)

RMSE Scores - 10 fold
* mean: 16139
* std: 10433


In [42]:
# find optimal features
param_grid = [{
    'feature_selection__k': list(range(1, len(feature_importances) + 1))
}]

rf_grid_final = GridSearchCV(full_pipeline, param_grid, cv=5,
                           scoring='neg_mean_squared_error', n_jobs=-1)

# fit model
rf_grid_final.fit(X_tr, y_train);

## Final Model

For our final model, we'll let the model find the best features rather than defining `k` on our own.

In [46]:
k = rf_grid_final.best_params_.get('feature_selection__k')

# define pipeline including column transformer, feature selection, and estimator
full_pipeline = Pipeline([
    ('col_transformer', col_tr),
    ('feature_selection', TopFeatureSelector(feature_importances, k)),
    ('rf_reg', RandomForestRegressor(**rf_grid.best_params_)),
])

In [47]:
# train model and print scores
train_model(full_pipeline, X_tr);

MAE = 2442
RMSE = 10517
R^2 = 0.711


In [48]:
# cross-validation
cv_rmse_scores(f'Random Forest - K{k}', full_pipeline.named_steps.rf_reg, X_train)

RMSE Scores - 10 fold
* mean: 16073
* std: 10240


In [49]:
# compare model performance
pd.DataFrame.from_dict(training_model_performance, orient='index').sort_values('rmse')

,rmse,std
Random Forest - Grid,16029,10302
Random Forest - K1,16073,10240
Random Forest - K20,16139,10433
Random Forest,18061,9984
Linear Regression,19726,8832
Decison Tree,24758,14149


In [51]:
# make final predictions
final_predictions = rf_grid_final.best_estimator_.predict(X_test)

# calculate RMSE score
final_mse = mean_squared_error(y_test, final_predictions)
final_rmse = np.sqrt(final_mse)
print('RMSE =', int(final_rmse))

RMSE = 21675


In [54]:
# compute 95% confidence interval for the test RMSE
confidence = 0.95
squared_errors = (final_predictions - y_test) ** 2
np.sqrt(stats.t.interval(confidence, len(squared_errors) - 1,
                         loc=squared_errors.mean(),
                         scale=stats.sem(squared_errors)))

/Users/miguel/anaconda3/envs/dev/lib/python3.7/site-packages/ipykernel_launcher.py:6: RuntimeWarning: invalid value encountered in sqrt
  


array([           nan, 31076.80351803])

In [56]:
# construct DataFrame with predictions & actuals
x_tst = X_test.copy()
x_tst['AcresBurned'] = y_test
x_tst['y_pred'] = final_predictions.astype('int')

# let's take a look
x_tst.sort_values('AcresBurned', ascending=False)

,AdminUnit,CalFireIncident,StartedYear,StartedMonth,Latitude,Longitude,sunsetTime,moonPhase,precipIntensity,temperatureHigh,temperatureLow,dewPoint,humidity,windSpeed,AcresBurned,y_pred
d40bee,US Forest Service - Los Padres National Forest,True,2017,December,34.415210,-119.091240,1512434820,0.56,0.0000,64.30,52.54,20.58,0.31,6.30,281893,76
5f6014,"Unified Command: CAL FIRE, Butte County Sherif...",True,2018,November,39.813400,-121.434700,1541638680,0.02,0.0006,70.30,45.21,2.06,0.12,6.47,153336,1512
d61261,CAL FIRE / USFS - El Dorado National Forest,True,2014,September,38.805437,-120.629714,1410660960,0.69,0.0000,91.81,70.25,36.69,0.22,1.40,97717,262
308e42,CAL FIRE Madera-Mariposa-Merced Unit,True,2017,July,37.617570,-120.213210,1500261780,0.76,0.0019,98.56,69.77,51.16,0.33,4.02,81826,94664
ec7941,CAL FIRE Sonoma-Lake-Napa Unit,True,2019,October,38.792458,-122.780053,1571880180,0.85,0.0000,75.20,66.60,31.50,0.26,8.83,77758,704
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3a5160,CAL FIRE/Riverside County Fire Department,True,2019,September,33.496633,-116.631106,1568167320,0.41,0.0006,73.17,51.46,44.82,0.55,6.31,10,200
3b2224,CAL FIRE Amador-El Dorado Unit,True,2017,August,38.344210,-120.722740,1503974400,0.25,0.0000,105.40,67.53,48.02,0.30,2.26,10,1649
b5b503,CAL FIRE / Butte County Fire,True,2019,April,39.839580,-121.957000,1556679720,0.88,0.0016,72.21,47.45,45.50,0.60,6.20,10,116
b17535,CAL FIRE / Riverside County Fire,True,2019,October,33.827979,-117.499619,1570757040,0.42,0.0001,80.02,61.67,24.48,0.37,7.14,9,105


In [58]:
# save model for later use
joblib.dump(rf_grid_final, '../models/final_random_forest_regressor_pipe')

['../models/final_random_forest_regressor_pipe']

### Findings

We knew from the data exploration step that this dataset isn't optimal as our target `AcresBurned` wasn't strongly correlated with any features. While our best features was `temperatureHigh`, it was dissapointing to see that the model chose to only include this feature in our final model, meaning that other features were mainly just noise.

On the bright side, we built a pipeline that can be used to transform our dataset, choose the optimal features, and make predictions on future data points.